# This is the example of KMC run for nasicon
By running the respective code block, the json file for kmc input should be generated. KMC will take the json as input paramter to execute the simulation

# 1. Local Cluster Expansion

## Required input 
- cif file of crystal structure: template_cif_fname
## Output
- lce.json
- lce_site.json

In [ ]:
from kmcpy.model import LocalClusterExpansion
mobile_ion_identifier_type="label"
mobile_ion_specie_1_identifier="Na1"
lce=LocalClusterExpansion(api=3)
lce.initialization3(mobile_ion_identifier_type=mobile_ion_identifier_type,mobile_ion_specie_1_identifier=mobile_ion_specie_1_identifier,cutoff_cluster=[8,6,0],cutoff_region=4,template_cif_fname='./file_before_kmc/EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif',convert_to_primitive_cell=True)
lce.to_json("lce.json")
lce.to_json("lce_site.json")

# 2. Fitting
Fit the Local Cluster Expansion model, data is from out KMC paper https://www.nature.com/articles/s41467-022-32190-7
## Required input 
- local cluster expansion correlation matrix, ekra, weight

    in the file\_before\_kmc/local\_cluster\_expansion
## Output
- fitting\_results.json

    fitting result of E_{KRA} and E_{site}
- (keci.txt)

    not input file for KMC, just for checking the parameters


In [ ]:
from kmcpy.fitting import Fitting
import numpy as np

local_cluster_expansion_fit = Fitting()

y_pred, y_true = local_cluster_expansion_fit.fit(alpha=1.5,max_iter=1000000,ekra_fname='file_before_kmc/local_cluster_expansion/e_kra.txt',keci_fname='keci.txt',
    weight_fname='file_before_kmc/local_cluster_expansion/weight.txt',corr_fname='file_before_kmc/local_cluster_expansion/correlation_matrix.txt',
    fit_results_fname='fitting_results.json')
print("fitting",y_pred, y_true )
local_cluster_expansion_fit = Fitting()

y_pred, y_true = local_cluster_expansion_fit.fit(alpha=1.5,max_iter=1000000,ekra_fname='file_before_kmc/local_cluster_expansion_site/e_kra.txt',keci_fname='keci.txt',
    weight_fname='file_before_kmc/local_cluster_expansion_site/weight.txt',corr_fname='file_before_kmc/local_cluster_expansion_site/correlation_matrix.txt',
    fit_results_fname='fitting_results_site.json')
print("fitting",y_pred, y_true )

# 3.Generate Events

## Required input 
- Cif file, the same as local cluster expansion

    path is defined in the prim_cif_name

## Output
- events.json

    store all events in the supercell 

- event\_kernel.csv

    event kernel for coupling between events

- 0th\_reference\_local\_env

    this is not the input for KMC. This is for checking. Should be the same as the local cluster expansion model.

- debug.log

    log file containing debug information. It is recommended to check the distance matrix

In [ ]:
mobile_ion_identifier_type="label"
mobile_ion_specie_1_identifier="Na1"
mobile_ion_specie_2_identifier="Na2"
prim_cif_name="file_before_kmc/EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif"
local_env_cutoff_dict={('Na+','Na+'):4,('Na+','Si4+'):4}
from kmcpy.event_generator import generate_events3

generate_events3(prim_cif_name=prim_cif_name,local_env_cutoff_dict=local_env_cutoff_dict,mobile_ion_identifier_type=mobile_ion_identifier_type,mobile_ion_specie_1_identifier=mobile_ion_specie_1_identifier,mobile_ion_specie_2_identifier=mobile_ion_specie_2_identifier,species_to_be_removed=["O2-","O","Zr4+","Zr"],distance_matrix_rtol=0.01,distance_matrix_atol=0.01,find_nearest_if_fail=False,convert_to_primitive_cell=True,export_local_env_structure=True,supercell_shape=[2,1,1],event_fname="events.json",event_kernal_fname='event_kernal.csv',verbosity="INFO")

# 4.Run KMC simulation
## Required input 
- input.json

    specify the parameter of kmc simulation
- initial_state.json

    initial occupation, typically obtained from a cluster expansion result
## Output

- standard output:
    
    contains information in each step

- \*.csv.gz file:

    contains the information of occupation, displacement, hopping counter, and results, respectively

In [ ]:

from kmcpy.io import InputSet,load_occ
from kmcpy.kmc import KMC
import numpy as np
api=3
inputset=InputSet.from_json("file_before_kmc/test_input_v3.json",api=3)

print(inputset._parameters.keys())
print(inputset._parameters["mc_results"])
inputset.parameter_checker()

inputset.set_parameter("occ",load_occ(fname=inputset._parameters["mc_results"],shape=inputset._parameters["supercell_shape"],select_sites=inputset._parameters["select_sites"],api=inputset.api,verbose=True))
inputset.set_parameter("use_numpy_random_kernel",False)
kmc = KMC(api=api)
events_initialized = kmc.initialization(**inputset._parameters) # v in 10^13 hz

# # step 2 compute the site kernal (used for kmc run)
kmc.load_site_event_list(inputset._parameters["event_kernel"])

# # step 3 run kmc
kmc_tracker=kmc.run_from_database(events=events_initialized,**inputset._parameters)


print(kmc_tracker.return_current_info())